In [1]:
from genetic_algorithm import *
import re
import json
import numpy as np
from numpy.random import default_rng
from collections import Counter
import copy
import matplotlib.pyplot as plt


## hexadecimal generator

In [2]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [3]:
## world size
world_size_x = 128
world_size_y = 128

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)


In [4]:
individuals_sum_dup = sum_duplicated_neurons(individuals)

## initial brain and position generator

In [16]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## brain generator
for individual in individuals_sum_dup:
    
    ## init brain neuron generator
    print(individual)
    ## preprocessing
    edges = individuals_sum_dup[individual]
    edges = [tuple(edges[i]) for i in edges]
    edges_no_weight = [i[:-1] for i in edges]
    
    G = nx.MultiDiGraph()
    G.add_edges_from(edges)
    
    out_list = list(set([i[1] for i in edges if 'out' in i[1]]))
    init_list = list(set([i[0] for i in edges if 'in' in i[0]]))

    # generate dict of paths
    dic_of_paths = generate_dict_of_paths(out_list, init_list, G)

    lis = filtered_neurons_paths(dic_of_paths)
    
    dic = append_weight_to_neurons_in_path(lis, edges_no_weight, edges)
    print(dic)
    
        
        

#     print(dic)
    print()

0
{'out1': {'mid0': 0.21895150768090438, 'out1': {'mid0': 2.139600183122234, 'in2': -0.4231344422401953}}, 'out2': {'mid0': 0.21895150768090438, 'mid2': {'in2': 1.9296200213642607, 'mid0': 2.97695711887685, 'in1': -0.8125743934075996}, 'out2': {'mid2': -0.20302151686250572}}, 'out0': {'mid0': 0.21895150768090438, 'mid2': {'in2': 1.9296200213642607, 'mid0': 2.97695711887685, 'in1': -0.8125743934075996}, 'out0': {'in1': -2.410865252556081, 'mid0': 1.7019380436441325, 'mid2': -3.42450785899588}}}

1
{'out1': {'mid1': -0.9023550677480728, 'out1': {'in1': -0.07446970853044407, 'mid1': -0.40103769265985045, 'in2': 1.0042423317564475}}, 'out2': {'mid1': -0.9023550677480728, 'out2': {'mid1': 1.6211200976651916}}, 'out0': {'out0': {'in2': 3.2983976804517017, 'in0': -3.3694491072791086}}}

2
{'out1': {'out1': {'in0': 0.3170456279566611, 'in2': -0.9643216847245536, 'in1': 0.8184343048985198}}, 'out2': {'mid1': {'in0': 3.811994506332977, 'mid2': 3.044712345490615}, 'mid2': -0.9413659853767962, 'ou

In [20]:
for i in dic:
    print(i)
    print(dic[i])
    print()

out2
{'mid0': 0.9901939361952762, 'mid1': -0.8966706992163574, 'mid2': {'in2': -0.6113840988860064, 'mid0': 3.5628261864794752, 'mid1': 3.4666259728368685}, 'out2': {'mid2': 3.6171524492598808}}

out1
{'mid0': 0.9901939361952762, 'mid1': -0.8966706992163574, 'mid2': {'in2': -0.6113840988860064, 'mid0': 3.5628261864794752, 'mid1': 3.4666259728368685}, 'out1': {'in0': -3.1268731878528917, 'in1': 0.03601403937128033, 'mid0': -3.0725469250724857, 'mid2': -3.168014649778727, 'in2': 0.36661071265069434}}



In [7]:
dic['out1']

{'mid0': 0.9901939361952762,
 'mid1': -0.8966706992163574,
 'mid2': {'in2': -0.6113840988860064,
  'mid0': 3.5628261864794752,
  'mid1': 3.4666259728368685},
 'out1': {'in0': -3.1268731878528917,
  'in1': 0.03601403937128033,
  'mid0': -3.0725469250724857,
  'mid2': -3.168014649778727,
  'in2': 0.36661071265069434}}

In [8]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## remove self loops
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

## brain generator
for individual in individuals_sum_dup_no_self_loop:
    
    ## init brain neuron generator
    
    ## preprocessing
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]

    remove_mid_with_no_predecessor(edges) 
    dic = weight_sum_preprocessing(edges)

    ## calculate weight sum
    mid_to_weight(dic)
    out_to_weight(dic)
       
    dic = {k:v for k, v in dic.items() if 'out' in k}    
    
    initial_dic_of_output_neurons[individual] = {}
    initial_dic_of_output_neurons[individual]['output'] = dic
    
    ## position
#     initial_dic_of_output_neurons[individual]['position'] = {}
#     initial_dic_of_output_neurons[individual]['position']['x'] = x[individual]
#     initial_dic_of_output_neurons[individual]['position']['y'] = y[individual] 
    
    ## position history
    initial_dic_of_output_neurons[individual]['history_position'] = [(x[individual], y[individual])]

In [9]:
initial_dic_of_output_neurons

{0: {'output': {'out0': -0.9941392171560546,
   'out1': -0.3995677412004998,
   'out2': -0.559667317504093},
  'history_position': [(90, 1)]},
 1: {'output': {'out0': -0.07093210467435282,
   'out1': -0.4384494215569999,
   'out2': 0.5525715141627122},
  'history_position': [(23, 123)]},
 2: {'output': {'out0': 0.9984096476650253,
   'out1': 0.8128864397791226,
   'out2': 0.9895769484375675},
  'history_position': [(119, 73)]},
 3: {'output': {'out1': -0.9999982503498028, 'out2': 0.9998047532689923},
  'history_position': [(121, 93)]}}

In [10]:
a = [(3, 0),(4,1),(5,2)]

In [11]:
def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

print(slope_intercept(3,0,2,1))

(-1.0, 3.0)


In [12]:
a

[(3, 0), (4, 1), (5, 2)]

In [13]:
if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
print(x)

3


In [14]:
a[-2][0]

4